## Imports

In [2]:
import os
import requests
from IPython.display import display, HTML
from tabulate import tabulate
import urllib.request


## Helper funcs. 

In [3]:

def fetch_all_data(url):
    """Fetch all data from the given URL."""
    print("retrieving data from", url)
    counter = 0 
    documents = []
    while url and counter < 1000:
        response = requests.get(url)
        data = response.json()
        documents.extend(data.get('value', []))
        url = data.get('@odata.nextLink', None)  # Get the next link if available
        counter += 1
    return documents

def truncate_string(s, max_length):
    """Truncate string to the max_length with ellipsis if needed."""
    return s if len(s) <= max_length else s[:max_length - 3] + '...'

def construct_url(base_url, filters=None, select=None, expand=None, skip=None, top=None, orderby=None, count=False):
    """Function to construct URL with OData filters"""
    query_params = []

    if filters:
        query_params.append(f"$filter={filters}")
    if select:
        query_params.append(f"$select={select}")
    if expand:
        query_params.append(f"$expand={expand}")
    if skip:
        query_params.append(f"$skip={skip}")
    if top:
        query_params.append(f"$top={top}")
    if orderby:
        query_params.append(f"$orderby={orderby}")
    if count:
        query_params.append(f"$count=true")

    if query_params:
        return f"{base_url}?" + "&".join(query_params)
    else:
        return base_url

## Letsgoo

In [4]:
base_url = 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/'
documentation_url = "https://opendata.tweedekamer.nl/documentatie/"

# Filters en params
filters = "Kamer eq 'tweede kamer' and year(datum) eq 2024"
select = None # Kan bijvoorbeeld zijn: 'id, naam, partij'
expand = "Verslag" # Bedoeld om child-data op te vragen, 
skip = None # Skip aantal records
top = None # Top aantal records
orderby = "Datum desc" # Orderby asc, desc
count = False # Count aantal records

# Maak url
combined_url = construct_url(base_url + "Vergadering", filters, select, expand, skip, top, orderby, count)
print(combined_url)

https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Vergadering?$filter=Kamer eq 'tweede kamer' and year(datum) eq 2024&$expand=Verslag&$orderby=Datum desc


In [5]:
vergadering_data = fetch_all_data(combined_url)

amount_of_rows = 10
data = []

for vergadering in vergadering_data:
    titel = truncate_string(vergadering['Titel'], 50)
    data.append([titel, vergadering['Datum'], vergadering['Zaal'], vergadering.get('Verslag')[-1].get('Soort')])

print(tabulate(data[:amount_of_rows], headers=['Titel', 'Datum', 'Zaal', 'Verslag soort']))

retrieving data from https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Vergadering?$filter=Kamer eq 'tweede kamer' and year(datum) eq 2024&$expand=Verslag&$orderby=Datum desc
Titel                                    Datum                      Zaal                      Verslag soort
---------------------------------------  -------------------------  ------------------------  ----------------
92e vergadering, donderdag 4 juli 2024   2024-07-04T00:00:00+02:00  Plenaire zaal             Tussenpublicatie
Mestbeleid                               2024-07-04T00:00:00+02:00  Thorbeckezaal             Tussenpublicatie
91e vergadering, woensdag 3 juli 2024    2024-07-03T00:00:00+02:00  Plenaire zaal             Tussenpublicatie
NAVO                                     2024-07-03T00:00:00+02:00  Thorbeckezaal             Tussenpublicatie
90e vergadering, donderdag 27 juni 2024  2024-06-27T00:00:00+02:00  Plenaire zaal             Tussenpublicatie
vmbo                                     2024-06

### Download verslagen

In [6]:
# We'll just be downloading 5 for now
for vergadering in vergadering_data[:5]:
    # Get (latest) verslag from vergadering
    verslag_id = vergadering['Verslag'][-1]['Id']
    verslag_url = f"{base_url}Verslag/{verslag_id}/resource"
    # Download the verslag URL in XML format
    urllib.request.urlretrieve(verslag_url, f'verslag-data/{verslag_id}.xml')


## Parse verslag XML to gain some insights

In [18]:
import os

import xml.etree.ElementTree as ET

# Get the path of the first XML file in the directory
xml_dir = './verslag-data/'
xml_files = os.listdir(xml_dir)
xml_file_path = os.path.join(xml_dir, xml_files[0])

print(f"Analyzing first XML file: {xml_file_path}")

# Parse the XML file
tree = ET.parse(xml_file_path)


Analyzing first XML file: ./verslag-data/0a69c26d-1b24-4012-b3c2-f827c0e65634.xml


FileNotFoundError: [Errno 2] No such file or directory: '//ns:alinea//text()'